In [10]:
import pandas as pd
from midiutil import MIDIFile

Let's import the files from net_data.

In [36]:
file_name='net_data/phone_on_4g.csv'

Put the CSV data into a Pandas data frame. Once it is in there, we will add columns

* Channel  - the stream for the connection.

* Time - the time of the event. Converted into epoch time by tshark

* Source  - the IP address that initiated the packet

* Target - the IP address that received the packet

* Data - the amount of data (in bytes) in the packet

In [13]:
phone_df = pd.read_csv(file_name, header=None)
phone_df.columns = ['Channel', 'Time', 'Source', 'Target', 'Data']
phone_df['Channel'] = phone_df['Channel'].replace('NaN', 0.0)
phone_df.head()

,Channel,Time,Source,Target,Data
0,NaN,1612192594,NaN,NaN,NaN
1,NaN,1612192601,192.168.1.1,224.0.0.1,NaN
2,NaN,1612192601,192.168.1.2,224.0.0.251,NaN
3,NaN,1612192602,192.168.1.6,224.0.0.251,NaN
4,NaN,1612192609,NaN,NaN,NaN


Get the first time stamp so that we have a 0 point in time to calculate from

In [14]:
start = phone_df['Time'].min()

Set the values for the MIDI variables

In [27]:
track    = 0
channel  = 0
time     = 0    # In beats
duration = 1    # In beats
tempo    = 60   # In BPM
volume   = 62  # 0-127, as per the MIDI standard

In [16]:
# One track, defaults to format 1 (tempo track is created automatically)
# set tracks to max number of streams connected
#num_tracks = int(phone_df['Channel'].max())
MyMIDI = MIDIFile(1)  
MyMIDI.addTempo(track, time, tempo)

We can't put an IP address here if we want to focus on a particular IP address. 

In [7]:
filtered_ip = ""

Let's iterate over the rows in the dataframe and filter by source or target. If the ip is being filtered, then the volume is louder than non-filtered ips. These are played at the time that they appear in the data stream. 

In [34]:
for _, row in phone_df.iterrows():
    if type(row['Source']) is not float and type(row['Target']):

        source = int(row['Source'].split('.')[0])
        _time = int(row['Time']-start)
        
        if filtered_ip is not None and source == filtered_ip:
            MyMIDI.addNote(0, 1, source, _time, 0.1, 60)
        else:
            MyMIDI.addNote(0, channel, source, _time, 0.1, 30)
        #create a note for the source

        #create a slight offset note for the target
        target = int(row['Target'].split('.')[0])
        targettime = int(_time + 1)

        if filtered_ip is not None and target == filtered_ip:
            MyMIDI.addNote(0, 1, target, _time, 0.1, 60)
        else:
            MyMIDI.addNote(0, channel, target, targettime, 0.1, 30)

Let's iterate over the data frame. Here we use the time difference between the time that the source/target pair have been seen. If they have been seen recently, then make it louder, If it is quieter, then it was seen a while ago. 

In [37]:
#let's set up a simple dictionary to hold the ips and the time for a simple calculation
ips = {}

for _, row in phone_df.iterrows():
    if type(row['Source']) is not float and type(row['Target']):
        

        source = row['Source']
        target = row['Target']
        
        if type(row['Channel']) is not float:
            channel = row['Channel']
        else:
            channel = 0
        
        #Have we seen it before?
        if source+target not in ips:
            ips[source+target] = {'time':0,'volume':60}
        else:
            #get the current last time. 
            tmp = ips[source+target]["time"]
            
            time_diff = (int(row['Time']) - ips[source+target]["time"])
            #if closer, then increase the volume
            if time_diff > ips[source+target]["time"]:
                if ips[source+target]["volume"] < 127:
                    ips[source+target]["volume"] += 1
            #if further away, then decrease the volume
            elif time_diff < ips[source+target]["time"]:
                if ips[source+target]["volume"] > 0:
                    ips[source+target]["volume"] -= 1
            
            ips[source+target] = {'time':time_diff,'volume':volume} 
                        
        _time = int(row['Time']-start)

        #create a note for the source
        MyMIDI.addNote(0, channel, source, _time, 0.1, volume)
        #create a slight offset note for the target
        
        targettime = int(_time + 1)
        MyMIDI.addNote(0, channel, target, targettime, 0.1, volume)

Let's write the file to disk now. The filename is from the CSV file. Maybe a todo is to put in time. 

In [35]:
with open(file_name[:-4]+"vol.mid", "wb") as output_file:
    MyMIDI.writeFile(output_file)

In [33]:
print(ips)

{'192.168.1.1224.0.0.1': {'time': 140, 'volume': 62}, '192.168.1.2224.0.0.251': {'time': 104, 'volume': 62}, '192.168.1.6224.0.0.251': {'time': 1612192740, 'volume': 62}, '52.97.133.226192.168.1.2': {'time': 1612192610, 'volume': 62}, '192.168.1.252.97.133.226': {'time': 1612192610, 'volume': 62}, '192.168.1.2129.102.3.227': {'time': 1612192610, 'volume': 62}, '129.102.3.227192.168.1.2': {'time': 58, 'volume': 62}, '192.168.1.246.4.77.182': {'time': 3, 'volume': 62}, '46.4.77.182192.168.1.2': {'time': 1612192785, 'volume': 62}, '192.168.1.2192.168.1.1': {'time': 20, 'volume': 62}, '192.168.1.1192.168.1.2': {'time': 1612192832, 'volume': 62}, '192.168.1.2216.58.212.234': {'time': 1612192644, 'volume': 62}, '216.58.212.234192.168.1.2': {'time': 1612192644, 'volume': 62}, '192.168.1.252.142.124.215': {'time': 1612192658, 'volume': 62}, '192.168.1.252.142.125.222': {'time': 1612192660, 'volume': 62}, '52.142.124.215192.168.1.2': {'time': 1612192712, 'volume': 62}, '52.142.125.222192.168.1.